# Quick Start Tutorial: Model Training

## Learning Objectives

In this tutorial you will learn:
1. How to design an observation set for your use case
2. How to materialize training data
3. How your ML training environment can consume training data

## Set up the prerequisites

Learning Objectives

In this section you will:
* connect to the remote featurebyte server
* import libraries
* learn the about catalogs
* activate a pre-built catalog

In [ ]:
import urllib.request

# install featurebyte package and download supporting library
!pip install --no-warn-conflicts featurebyte
urllib.request.urlretrieve("https://raw.githubusercontent.com/featurebyte/featurebyte-hosted-tutorials/main/tutorials/notebooks/prebuilt_catalogs.py", "prebuilt_catalogs.py")

In [1]:
# library imports
import pandas as pd
import numpy as np
import random

# load the featurebyte SDK
import featurebyte as fb

# replace <api_token> with your API token you receieved after registering
fb.register_tutorial_api_token("<api_token>")

# define the database name for this tutorial
TUTORIAL_DATABASE = "TUTORIAL_DATASETS"

14:02:27 | INFO     | Using configuration file at: /Users/smillet/.featurebyte/config.yaml
14:02:27 | WARNING  | No valid profile specified. Update config file or specify valid profile name with "use_profile".
14:02:27 | INFO     | Using profile: tutorial
14:02:27 | INFO     | Using configuration file at: /Users/smillet/.featurebyte/config.yaml
14:02:27 | INFO     | Active profile: tutorial (https://tutorials.featurebyte.com/api/v1)
14:02:27 | WARNING  | Remote SDK version (0.3.0.dev136) is different from local (0.4.1). Update local SDK to avoid unexpected behavior.
14:02:27 | INFO     | No catalog activated.
14:02:27 | INFO     | 2 feature lists, 9 features deployed


### Create a pre-built catalog for this tutorial, with the data, metadata, and features already set up

Note that creating a pre-built catalog is not a step you will do in real-life. This is a function specific to this quick-start tutorial to quickly skip over many of the preparatory steps and get you to a point where you can materialize features.

In a real-life project you would do data modeling, declaring the tables, entities, and the associated metadata. This would not be a frequent task, but forms the basis for best-practice feature engineering.

In [2]:
# get the functions to create a pre-built catalog
from prebuilt_catalogs import *

# create a new catalog for this tutorial
catalog = create_tutorial_catalog(PrebuiltCatalog.QuickStartModelTraining)

Cleaning up existing tutorial catalogs
Building a quick start catalog for model training named [quick start model training 20230726:1402]
Creating new catalog
Catalog created


14:02:45 | INFO     | Catalog activated: quick start model training 20230726:1402


Registering the source tables
Registering the entities
Tagging the entities to columns in the data tables
Populating the feature store with example features
Done! |████████████████████████████████████████| 100% in 6.5s (0.16%/s)         
Done! |████████████████████████████████████████| 100% in 6.5s (0.16%/s)         
Done! |████████████████████████████████████████| 100% in 6.4s (0.16%/s)         
Loading Feature(s) |████████████████████████████████████████| 1/1 [100%] in 0.5s
Done! |████████████████████████████████████████| 100% in 6.5s (0.16%/s)         
Done! |████████████████████████████████████████| 100% in 6.5s (0.16%/s)         
Done! |████████████████████████████████████████| 100% in 10.3s (0.10%/s)        
Done! |████████████████████████████████████████| 100% in 9.8s (0.10%/s)         
Done! |████████████████████████████████████████| 100% in 6.7s (0.15%/s)         
Loading Feature(s) |████████████████████████████████████████| 1/1 [100%] in 0.5s
Done! |██████████████████████████

### Example: Create views from tables in the Catalog

In [3]:
# create the views
grocery_customer_view = catalog.get_view("GROCERYCUSTOMER")
grocery_invoice_view = catalog.get_view("GROCERYINVOICE")
grocery_items_view = catalog.get_view("INVOICEITEMS")
grocery_product_view = catalog.get_view("GROCERYPRODUCT")

## Create an observation set for your use case

Learning Objectives

In this section you will learn:
* the purpose of observation sets
* the relationship between entities, point in time, and observation sets
* how to design an observation set suitable for training data

### Case Study: Predicting Customer Spend

Your chain of grocery stores wants to target market customers immediately after each purchase. As one step in this marketing campaign, they want to predict future customer spend in the 14 days after a purchase.

### Concept: Materialization

A feature in FeatureByte is defined by the logical plan for its computation. The act of computing the feature is known as Feature Materialization.

The materialization of features is made on demand to fulfill historical requests, whereas for prediction purposes, feature values are generated through a batch process called a "Feature Job". The Feature Job is scheduled based on the defined settings associated with each feature.

### Concept: Observation set

An observation set combines entity key values and historical points-in-time, for which you wish to materialize feature values.

The observation set can be a pandas DataFrame or an ObservationTable object representing an observation set in the feature store. An accepted serving name must be used for the column containing the entity values. The column containing points-in-time must be labelled "POINT-IN-TIME" and the point-in-time timestamps should be in UTC.

### Concept: Point in time

A point-in-time for a feature refers to a specific moment in the past with which the feature's values are associated.

It is a crucial aspect of historical feature serving, which allows machine learning models to make predictions based on historical data. By providing a point-in-time, a feature can be used to train and test models on past data, enabling them to make accurate predictions for similar situations in the future.

### Case Study: Predicting Customer Spend

Your chain of grocery stores wants to target market customers immediately after each purchase. As one step in this marketing campaign, they want to predict future customer spend in the 14 days after a purchase.

In [4]:
# get the feature list for the target feature
customer_target_list = catalog.get_feature_list("TargetFeature")

# display details about the target feature
display(customer_target_list.list_features())

Loading Feature(s) |████████████████████████████████████████| 1/1 [100%] in 0.5s


,id,name,version,dtype,readiness,online_enabled,tables,primary_tables,entities,primary_entities,created_at,is_default
0,64c16047c04bc9b7b80aee55,Target,V230726,FLOAT,PRODUCTION_READY,False,[GROCERYINVOICE],[GROCERYINVOICE],[grocerycustomer],[grocerycustomer],2023-07-26 18:04:57.100,True


In [5]:
# create a large observation table from a view

# filter the view to exclude points in time that won't have data for historical windows
filter = (grocery_invoice_view["Timestamp"] >= pd.to_datetime("2022-04-01")) & (
    grocery_invoice_view["Timestamp"] < pd.to_datetime("2023-04-01")
)
observation_set_view = grocery_invoice_view[filter].copy()

# create a new observation table
observation_table = observation_set_view.create_observation_table(
    name="10,000 Customers immediately after each purchase from May-22 to Mar-23",
    sample_rows=10000,
    columns=["Timestamp", "GroceryCustomerGuid"],
    columns_rename_mapping={
        "Timestamp": "POINT_IN_TIME",
        "GroceryCustomerGuid": "GROCERYCUSTOMERGUID",
    },
)

# if the observation table isn't too large, you can materialize it
display(observation_table.to_pandas())

Done! |████████████████████████████████████████| 100% in 6.5s (0.16%/s)         


,POINT_IN_TIME,GROCERYCUSTOMERGUID
0,2022-04-08 13:10:00,5c96089d-95f7-4a12-ab13-e082836253f1
1,2022-04-11 11:49:05,5c96089d-95f7-4a12-ab13-e082836253f1
2,2022-04-18 15:29:26,5c96089d-95f7-4a12-ab13-e082836253f1
3,2022-05-20 13:03:26,5c96089d-95f7-4a12-ab13-e082836253f1
4,2022-05-25 13:01:50,5c96089d-95f7-4a12-ab13-e082836253f1
...,...,...
9995,2022-11-18 15:30:46,e2034c31-e304-4a0a-983f-4f08ca79c528
9996,2022-11-29 13:06:08,e2034c31-e304-4a0a-983f-4f08ca79c528
9997,2022-12-21 14:12:54,e2034c31-e304-4a0a-983f-4f08ca79c528
9998,2022-12-18 17:40:11,e2034c31-e304-4a0a-983f-4f08ca79c528


## Materialize Training Data

Learning Objectives

In this section you will learn:
* how to create historical training data
* how to merge target and features

### Example: Get historical values

In [6]:
# list the feature lists
display(catalog.list_feature_lists())

,id,name,num_feature,status,deployed,readiness_frac,online_frac,tables,entities,primary_entities,created_at
0,64c16047c04bc9b7b80aee51,Features,8,DRAFT,False,1.0,0.0,"[GROCERYCUSTOMER, GROCERYINVOICE, INVOICEITEMS...","[grocerycustomer, frenchstate]",[grocerycustomer],2023-07-26 18:05:09.014
1,64c1604fc04bc9b7b80aee58,TargetFeature,1,DRAFT,False,1.0,0.0,[GROCERYINVOICE],[grocerycustomer],[grocerycustomer],2023-07-26 18:05:04.268


In [7]:
# get the feature list
feature_list = catalog.get_feature_list("Features")

Loading Feature(s) |████████████████████████████████████████| 8/8 [100%] in 0.6s


In [8]:
# Compute the historical feature table
training_table_features = feature_list.compute_historical_feature_table(
    observation_table,
    historical_feature_table_name="customer training table - invoices Apr-22 to Mar-23 - features only",
)

# display the training data
display(training_table_features.to_pandas())

Done! |████████████████████████████████████████| 100% in 29.1s (0.03%/s)        


,POINT_IN_TIME,GROCERYCUSTOMERGUID,StatePopulation,StateAvgInvoiceAmount_28d,StateMeanLatitude,StateMeanLongitude,CustomerInventoryStability_14d28d,CustomerStateSimilarity_28d,CustomerSpend_28d,CustomerAvgInvoiceAmount_28d
0,2022-07-06 15:10:00,fb92d601-9f59-41fb-97fe-79d627d95bd8,181,17.655019,48.738534,2.239697,0.476731,0.521609,33.99,6.798000
1,2022-04-10 17:08:18,28fb4a20-554a-4823-a8b7-d810ebe56b9d,56,19.730168,43.452577,5.848259,0.951307,0.405683,115.31,12.812222
2,2022-07-05 16:03:08,7a024068-3f99-4114-9d90-3a61f679be51,181,17.605217,48.738534,2.239697,0.851081,0.609999,80.61,16.122000
3,2022-08-29 08:36:16,11465751-0f71-413d-b8c9-90b5a8f26c5f,29,18.877708,50.668117,2.918225,0.000000,0.105503,4.50,4.500000
4,2022-12-21 13:44:27,8082a592-79ed-46de-9204-f605461f0ec4,28,23.631509,50.665263,2.908103,0.919837,0.777275,192.63,24.078750
...,...,...,...,...,...,...,...,...,...,...
9995,2022-08-16 14:06:34,568e0d90-106d-4cbb-828d-32ebca059d48,25,15.711364,44.718312,-0.478629,0.809693,0.509796,111.41,12.378889
9996,2022-12-30 11:19:11,0b2de469-1034-4a6a-ac20-f040e7e40cbd,13,17.542162,47.182230,4.394402,0.941384,0.731648,224.56,11.228000
9997,2022-08-07 15:59:08,94127b9f-1366-4bbe-afea-7cd77225da52,8,17.573750,48.789776,5.855939,0.880385,0.905017,221.15,15.796429
9998,2022-04-04 11:46:18,257d9e61-5f0f-457c-9434-0e7376fce954,2,11.871111,46.307372,3.361389,0.766032,0.900919,46.97,7.828333


### Example: Get target values

When target values use aggregates or time offsets, you first need to offset the point in time by the time window.

In [9]:
# add 14 days to the timestamps in the observation set
observation_set_target = observation_table.to_pandas().copy()
observation_set_target["POINT_IN_TIME"] = observation_set_target["POINT_IN_TIME"] + pd.DateOffset(
    days=14
)

display(observation_set_target)

,POINT_IN_TIME,GROCERYCUSTOMERGUID
0,2022-04-22 13:10:00,5c96089d-95f7-4a12-ab13-e082836253f1
1,2022-04-25 11:49:05,5c96089d-95f7-4a12-ab13-e082836253f1
2,2022-05-02 15:29:26,5c96089d-95f7-4a12-ab13-e082836253f1
3,2022-06-03 13:03:26,5c96089d-95f7-4a12-ab13-e082836253f1
4,2022-06-08 13:01:50,5c96089d-95f7-4a12-ab13-e082836253f1
...,...,...
9995,2022-12-02 15:30:46,e2034c31-e304-4a0a-983f-4f08ca79c528
9996,2022-12-13 13:06:08,e2034c31-e304-4a0a-983f-4f08ca79c528
9997,2023-01-04 14:12:54,e2034c31-e304-4a0a-983f-4f08ca79c528
9998,2023-01-01 17:40:11,e2034c31-e304-4a0a-983f-4f08ca79c528


In [10]:
# Materialize the target feature using get historical features
training_data_target = customer_target_list.compute_historical_features(observation_set_target)

# remove the offset from the point in time column
training_data_target["POINT_IN_TIME"] = training_data_target["POINT_IN_TIME"] - pd.DateOffset(
    days=14
)

display(training_data_target)

Done! |████████████████████████████████████████| 100% in 12.9s (0.08%/s)        
Done! |████████████████████████████████████████| 100% in 6.5s (0.16%/s)         


,POINT_IN_TIME,GROCERYCUSTOMERGUID,Target
0,2022-04-08 13:10:00,5c96089d-95f7-4a12-ab13-e082836253f1,107.77
1,2022-04-11 11:49:05,5c96089d-95f7-4a12-ab13-e082836253f1,79.24
2,2022-04-18 15:29:26,5c96089d-95f7-4a12-ab13-e082836253f1,20.97
3,2022-05-20 13:03:26,5c96089d-95f7-4a12-ab13-e082836253f1,125.07
4,2022-05-25 13:01:50,5c96089d-95f7-4a12-ab13-e082836253f1,134.36
...,...,...,...
9995,2022-11-18 15:30:46,e2034c31-e304-4a0a-983f-4f08ca79c528,64.91
9996,2022-11-29 13:06:08,e2034c31-e304-4a0a-983f-4f08ca79c528,72.67
9997,2022-12-21 14:12:54,e2034c31-e304-4a0a-983f-4f08ca79c528,63.05
9998,2022-12-18 17:40:11,e2034c31-e304-4a0a-983f-4f08ca79c528,143.43


### Example: Merging materialized values for features and target

In [11]:
# merge training data features and training data target
training_data = training_table_features.to_pandas()
training_data = training_data.merge(
    training_data_target, on=["GROCERYCUSTOMERGUID", "POINT_IN_TIME"]
)

display(training_data)

,POINT_IN_TIME,GROCERYCUSTOMERGUID,StatePopulation,StateAvgInvoiceAmount_28d,StateMeanLatitude,StateMeanLongitude,CustomerInventoryStability_14d28d,CustomerStateSimilarity_28d,CustomerSpend_28d,CustomerAvgInvoiceAmount_28d,Target
0,2022-07-06 15:10:00,fb92d601-9f59-41fb-97fe-79d627d95bd8,181,17.655019,48.738534,2.239697,0.476731,0.521609,33.99,6.798000,15.83
1,2022-04-10 17:08:18,28fb4a20-554a-4823-a8b7-d810ebe56b9d,56,19.730168,43.452577,5.848259,0.951307,0.405683,115.31,12.812222,120.77
2,2022-07-05 16:03:08,7a024068-3f99-4114-9d90-3a61f679be51,181,17.605217,48.738534,2.239697,0.851081,0.609999,80.61,16.122000,42.63
3,2022-08-29 08:36:16,11465751-0f71-413d-b8c9-90b5a8f26c5f,29,18.877708,50.668117,2.918225,0.000000,0.105503,4.50,4.500000,11.98
4,2022-12-21 13:44:27,8082a592-79ed-46de-9204-f605461f0ec4,28,23.631509,50.665263,2.908103,0.919837,0.777275,192.63,24.078750,106.17
...,...,...,...,...,...,...,...,...,...,...,...
9995,2022-08-16 14:06:34,568e0d90-106d-4cbb-828d-32ebca059d48,25,15.711364,44.718312,-0.478629,0.809693,0.509796,111.41,12.378889,81.46
9996,2022-12-30 11:19:11,0b2de469-1034-4a6a-ac20-f040e7e40cbd,13,17.542162,47.182230,4.394402,0.941384,0.731648,224.56,11.228000,35.77
9997,2022-08-07 15:59:08,94127b9f-1366-4bbe-afea-7cd77225da52,8,17.573750,48.789776,5.855939,0.880385,0.905017,221.15,15.796429,60.34
9998,2022-04-04 11:46:18,257d9e61-5f0f-457c-9434-0e7376fce954,2,11.871111,46.307372,3.361389,0.766032,0.900919,46.97,7.828333,134.34


## Consuming training data

Learning Objectives

In this section you will learn:
* how to save a training file
* how to use a pandas data frame

### Example: Save the training data to a file

In [12]:
# save training data as a csv file
training_data.to_csv("training_data.csv", index=False)

In [13]:
# save the training file as a parquet file
training_data.to_parquet("training_data.parquet")

### Example: Training a scikit learn model

Note that you will need to install scikit learn https://scikit-learn.org/stable/install.html

In [14]:
# EDA on the training data
training_data.describe()

,StatePopulation,StateAvgInvoiceAmount_28d,StateMeanLatitude,StateMeanLongitude,CustomerInventoryStability_14d28d,CustomerStateSimilarity_28d,CustomerSpend_28d,CustomerAvgInvoiceAmount_28d,Target
count,10000.00000,9995.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,9619.000000,10000.000000
mean,79.42100,18.226662,45.453526,3.181993,0.759764,0.591044,136.814154,18.118967,84.433821
std,75.67579,3.636491,9.376102,8.804621,0.298374,0.219834,124.459966,14.383738,72.881828
min,1.00000,4.593333,-12.713308,-50.017299,0.000000,0.000000,0.000000,0.000000,0.000000
25%,14.00000,16.548856,44.663768,2.237559,0.728429,0.481283,44.025000,8.506894,30.372500
50%,33.00000,17.817885,48.211446,2.241215,0.884818,0.629658,98.665000,14.671667,63.285000
75%,180.00000,20.135909,48.739485,5.054081,0.945965,0.744580,196.252500,23.268750,119.342500
max,183.00000,35.424286,50.669452,45.189819,1.000000,1.000000,801.030000,182.345000,541.140000


In [15]:
# do any columns in the training data contain missing values?
training_data.isna().any()

POINT_IN_TIME                        False
GROCERYCUSTOMERGUID                  False
StatePopulation                      False
StateAvgInvoiceAmount_28d             True
StateMeanLatitude                    False
StateMeanLongitude                   False
CustomerInventoryStability_14d28d    False
CustomerStateSimilarity_28d          False
CustomerSpend_28d                    False
CustomerAvgInvoiceAmount_28d          True
Target                               False
dtype: bool

In [16]:
! pip install scikit-learn


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip


In [17]:
# use sklearn to train a random forest regression model on the training data
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    training_data.drop(columns=["GROCERYCUSTOMERGUID", "POINT_IN_TIME"]),
    training_data["Target"],
    test_size=0.2,
    random_state=42,
)

# train the model
model = HistGradientBoostingRegressor()
model.fit(X_train, y_train)

# get predictions
y_pred = model.predict(X_test)

# calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean squared error: ", mse)

# save the model
import joblib

joblib.dump(model, "model.pkl")

Mean squared error:  3.175295058371279


['model.pkl']

## Next Steps

Now that you've completed the quick-start feature engineering tutorial, you can put your knowledge into practice or learn more:<br>
1. Learn more about materializing features via the "Deep Dive Materializing Features" tutorial
2. Put your knowledge into practice by creating features in the "credit card dataset feature engineering playground" or "healthcare dataset feature engineering playground" workspaces
3. Learn more about feature governance via the "Quick Start Feature Governance" tutorial